### Web Search Tool

In [4]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool 
def web_search(query:str) -> Dict[str,Any]:
  """Search the web for information"""
  return tavily_client.search(query)

web_search.invoke("Who is the current Mayor of San Fransisco")

{'query': 'Who is the current Mayor of San Fransisco',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Mayor_of_San_Francisco',
   'title': 'Mayor of San Francisco - Wikipedia',
   'content': 'The current mayor is Democrat Daniel Lurie.',
   'score': 0.9995234,
   'raw_content': None},
  {'url': 'https://apnews.com/article/san-francisco-new-mayor-liberal-city-81ea0a7b37af6cbb68aea7ef5cc6a4f0',
   'title': "San Francisco's new mayor is starting to unite the fractured city",
   'content': 'San Francisco Mayor Daniel Lurie, a political newcomer and Levi Strauss heir, has marked his first 100 days with a hands-on, business-friendly approach.',
   'score': 0.99648935,
   'raw_content': None},
  {'url': 'https://www.sf.gov/departments--office-mayor',
   'title': 'Office of the Mayor - SF.gov',
   'content': 'Daniel Lurie is the 46th Mayor of the City and County of San Francisco.',
   'score': 0.9951703,
   'raw_content': None}

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

from langchain.agents import create_agent

agent = create_agent(
  model=model,
  tools=[web_search]
)

response = agent.invoke(
  {"messages":["Who is the mayor of San Fransisco"]}
)

print(response['messages'][-1].content)

The current mayor of San Francisco is Daniel Lurie. He is the 46th mayor of the city and county.


### Short - term Memory 

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
  model=model,
  checkpointer=InMemorySaver(),
)

In [7]:
from langchain.messages import HumanMessage
question = HumanMessage(content="Hello my name is Sean and my favorite color is Green")

config = {"configurable":{"thread_id":"1"}}

response = agent.invoke(
  {"messages":[question]},
  config
)

from pprint import pprint

print(response)

{'messages': [HumanMessage(content='Hello my name is Sean and my favorite color is Green', additional_kwargs={}, response_metadata={}, id='10ac5eba-ddb6-4121-bafa-e5147d0cff78'), AIMessage(content="Hi Sean! It's nice to meet you. Green is a fantastic color, often associated with nature, growth, and tranquility. It's a great choice!\n\nWhat do you like about green, Sean?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bdc55-9691-7f72-8c1b-30082dab1702-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 44, 'total_tokens': 56, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='What is my favorite color ?', additional_kwargs={}, response_metadata={}, id='52cc64ac-a309-4513-b4b7-061c947fe7c9'), AIMessage(content='Your favorite color is **Green**, Sean! You told me that in your very first message. 😊', add

In [8]:
question2 = HumanMessage(content="What is my favorite color ?")

res = agent.invoke(
  {"messages":[question2]},
  config
)

pprint(response)

{'messages': [HumanMessage(content='Hello my name is Sean and my favorite color is Green', additional_kwargs={}, response_metadata={}, id='10ac5eba-ddb6-4121-bafa-e5147d0cff78'),
              AIMessage(content="Hi Sean! It's nice to meet you. Green is a fantastic color, often associated with nature, growth, and tranquility. It's a great choice!\n\nWhat do you like about green, Sean?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bdc55-9691-7f72-8c1b-30082dab1702-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 44, 'total_tokens': 56, 'input_token_details': {'cache_read': 0}}),
              HumanMessage(content='What is my favorite color ?', additional_kwargs={}, response_metadata={}, id='52cc64ac-a309-4513-b4b7-061c947fe7c9'),
              AIMessage(content='Your favorite color is **Green**, Sean! You told m

### 